## Image-based AI - Predicting Age

We show how AI models can make predictions from images, and show image-based explanations (salience maps). 

The case study we use is the prediction of age from images of the face. The distinction between predicted age and biological age (from various prediction mediums) is often considered to be a biomarker with relevance to health -- for a recent example of this type of study, see https://pubmed.ncbi.nlm.nih.gov/38190540/. 

Here, we focus only on the readily obtainable images of the face. We work from the FaceXFormer pre-trained model: 


In [ ]:
# Install requirements
pip install torch==2.0.1+cu117 torchvision==0.15.2+cu117 torchaudio==2.0.2+cu117 --extra-index-url https://download.pytorch.org/whl/cu117
pip install -r requirements.txt

### P.S. Brain age prediction 

For a more technical and involved case study, see https://www.medrxiv.org/content/10.1101/2022.09.06.22279594v2.full.pdf for an example of age prediction from brain MRI images. The source code and model weights are available at https://github.com/westman-neuroimaging-group/brainage-prediction-mri . 

